In [1]:
# Import necessary libraries
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# Load the California housing dataset
housing = fetch_california_housing()
X = housing.data
y = housing.target

In [3]:
# Split the dataset into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Standardize our features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
# Feature selection -  using SelectKBest to select the top 5 features
k = 5  # Choose the number of top features to select
selector = SelectKBest(score_func=f_regression, k=k)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

In [8]:
# Now  use Recursive Feature Elimination (RFE) to select the top 5 features
# RFE uses an external estimator to weigh features (we'll use LinearRegression)
model = LinearRegression()
selector_rfe = RFE(model, n_features_to_select=k)
X_train_rfe = selector_rfe.fit_transform(X_train_scaled, y_train)
X_test_rfe = selector_rfe.transform(X_test_scaled)

In [10]:
# Dimensionality reduction - use PCA to reduce our feature space to 5 dimensions
pca = PCA(n_components=k)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [11]:
# Initialize our Linear Regression model
lr = LinearRegression()

In [12]:
# Train and evaluate the model using the selected features from SelectKBest
lr.fit(X_train_selected, y_train)
predictions_selected = lr.predict(X_test_selected)
mse_selected = mean_squared_error(y_test, predictions_selected)

In [13]:
# Train and evaluate the model using the selected features from RFE
lr.fit(X_train_rfe, y_train)
predictions_rfe = lr.predict(X_test_rfe)
mse_rfe = mean_squared_error(y_test, predictions_rfe)

In [14]:
# Train and evaluate the model using the principal components from PCA
lr.fit(X_train_pca, y_train)
predictions_pca = lr.predict(X_test_pca)
mse_pca = mean_squared_error(y_test, predictions_pca)

In [15]:

# Print the MSE for each feature selection method
print(f'MSE with SelectKBest: {mse_selected}')
print(f'MSE with RFE: {mse_rfe}')
print(f'MSE with PCA: {mse_pca}')

MSE with SelectKBest: 0.6382565441555919
MSE with RFE: 0.5667695170781499
MSE with PCA: 0.7431025684653442
